In [2]:
import taichi as ti
from numpy import linspace, array
from scipy.interpolate import pchip_interpolate

In [11]:
import taichi as ti
import math

ti.init(arch=ti.cpu)

n = 400  # pixels of | orientation
aspect_ratio = 1
max_iterations = 300  # The more this value is, the figure is finer
pixels = ti.field(ti.f64, shape=(600, 600))

@ti.func
def complex_sqr(z):
    return ti.Vector([z[0]**2 - z[1]**2, z[1] * z[0] * 2])

@ti.kernel
def paint(cx: ti.f64, cy: ti.f64, r:ti.f64):
    for i, j in pixels:  # Parallized over all pixels
        c = ti.Vector([-0.8, 0.2])
        z = ti.Vector([i / n * r - aspect_ratio*0.5*r + cx,
                       j / n * r - 0.5*r + cy])
        iterations = 0
        while z.norm() < 3:
            z = complex_sqr(z) + c
            iterations += 1
        pixels[i, j] = 1 - iterations / max_iterations

[Taichi] Starting on arch=x64


array([[0.88666666, 0.84333336, 0.92666668, ..., 0.84666669, 0.85333335,
        0.85666668],
       [0.87666667, 0.86000001, 0.93000001, ..., 0.85000002, 0.85666668,
        0.85666668],
       [0.88999999, 0.92000002, 0.93000001, ..., 0.85000002, 0.85333335,
        0.85666668],
       ...,
       [0.92666668, 0.92666668, 0.92666668, ..., 0.98666668, 0.98666668,
        0.98666668],
       [0.92666668, 0.92666668, 0.92666668, ..., 0.98666668, 0.98666668,
        0.98666668],
       [0.92666668, 0.92666668, 0.92666668, ..., 0.98666668, 0.98666668,
        0.98666668]])

In [7]:
def generate_mandelbrot(canvas):
    for x in range(CANVAS_WIDTH):
        for y in range(CANVAS_HEIGHT):
            zx, zy = x * 3.5 / CANVAS_WIDTH - 2.5, y * 2.0 / CANVAS_HEIGHT - 1.0
            c = zx + zy * 1j
            z = c
            color = 0
            for i in range(256):
                if abs(z) > 2.0:
                    color = i
                    break 
                z = z * z + c
            color_hex = f"#{color:02x}{color:02x}{color:02x}"
            canvas.create_rectangle(x, y, x + 1, y + 1, fill=color_hex, outline=color_hex)

In [13]:
import random
class Example(tk.Frame):
    def __init__(self, root):
        tk.Frame.__init__(self, root)
        self.canvas = tk.Canvas(self, width=600, height=600, background="white")
        self.xsb = tk.Scrollbar(self, orient="horizontal", command=self.canvas.xview)
        self.ysb = tk.Scrollbar(self, orient="vertical", command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=self.ysb.set, xscrollcommand=self.xsb.set)
        self.canvas.configure(scrollregion=(0,0,1000,1000))

        self.xsb.grid(row=1, column=0, sticky="ew")
        self.ysb.grid(row=0, column=1, sticky="ns")
        self.canvas.grid(row=0, column=0, sticky="nsew")
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)

        #Plot some rectangles
        # for n in range(50):
        #     x0 = random.randint(0, 900)
        #     y0 = random.randint(50, 900)
        #     x1 = x0 + random.randint(50, 100)
        #     y1 = y0 + random.randint(50,100)
        #     color = ("red", "orange", "yellow", "green", "blue")[random.randint(0,4)]
        #     self.canvas.create_rectangle(x0,y0,x1,y1, outline="black", fill=color, activefill="black", tags=n)
        # self.canvas.create_text(50,10, anchor="nw", text="Click and drag to move the canvas\nScroll to zoom.")

        # This is what enables using the mouse:
        self.canvas.bind("<ButtonPress-1>", self.move_start)
        self.canvas.bind("<B1-Motion>", self.move_move)
        #linux scroll
        self.canvas.bind("<Button-4>", self.zoomerP)
        self.canvas.bind("<Button-5>", self.zoomerM)
        #windows scroll
        self.canvas.bind("<MouseWheel>",self.zoomer)

    #move
    def move_start(self, event):
        self.canvas.scan_mark(event.x, event.y)
    def move_move(self, event):
        self.canvas.scan_dragto(event.x, event.y, gain=1)

    #windows zoom
    def zoomer(self,event):
        if (event.delta > 0):
            self.canvas.scale("all", event.x, event.y, 1.1, 1.1)
        elif (event.delta < 0):
            self.canvas.scale("all", event.x, event.y, 0.9, 0.9)
        self.canvas.configure(scrollregion = self.canvas.bbox("all"))

    #linux zoom
    def zoomerP(self,event):
        self.canvas.scale("all", event.x, event.y, 1.1, 1.1)
        self.canvas.configure(scrollregion = self.canvas.bbox("all"))
    def zoomerM(self,event):
        self.canvas.scale("all", event.x, event.y, 0.9, 0.9)
        self.canvas.configure(scrollregion = self.canvas.bbox("all"))

if __name__ == "__main__":
    root = tk.Tk()
    Example(root).pack(fill="both", expand=True)
    root.mainloop()

In [16]:
import tkinter as tk
import numpy as np

class MandelbrotViewer:
    def __init__(self, master):
        self.master = master
        self.canvas = tk.Canvas(master, width=800, height=800)
        self.canvas.pack()
        self.xmin, self.xmax, self.ymin, self.ymax = -2.5, 1.5, -1.5, 1.5
        self.draw_mandelbrot()
        self.canvas.bind("<MouseWheel>", self.zoom)

    def draw_mandelbrot(self):
        self.canvas.delete("all")
        for x in range(800):
            for y in range(800):
                zx, zy = self.xmin + x * (self.xmax - self.xmin) / 800, self.ymin + y * (self.ymax - self.ymin) / 800
                c = complex(zx, zy)
                z = complex(0, 0)
                for i in range(256):
                    z = z * z + c
                    if abs(z) > 2:
                        break
                if i == 255:
                    self.canvas.create_rectangle(x, y, x + 1, y + 1, fill="#000000")
                else:
                    self.canvas.create_rectangle(x, y, x + 1, y + 1, fill="#{:02x}{:02x}{:02x}".format(i, i, i))

    def zoom(self, event):
        if event.delta > 0:
            self.xmin, self.xmax, self.ymin, self.ymax = self.xmin + (self.xmax - self.xmin) / 4, self.xmax - (self.xmax - self.xmin) / 4, self.ymin + (self.ymax - self.ymin) / 4, self.ymax - (self.ymax - self.ymin) / 4
        else:
            self.xmin, self.xmax, self.ymin, self.ymax = self.xmin - (self.xmax - self.xmin) / 2, self.xmax + (self.xmax - self.xmin) / 2, self.ymin - (self.ymax - self.ymin) / 2, self.ymax + (self.ymax - self.ymin) / 2
        self.draw_mandelbrot()

root = tk.Tk()
app = MandelbrotViewer(root)
root.mainloop()